Analyses pre-registered for in-lab version of study 3d (reported in the appendix accompanying the submitted manuscript as study 3d).

Pre-registration: https://osf.io/de935

In [1]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re
from scipy import stats
import statsmodels.stats.api as sms
from pyspan.utils import *
from pyspan.plurals.analysis import *
assert not mturk
from pyspan.plurals.preprocess import *
from pyspan.plurals.utils import *

/Users/sabinasloman/Box/LoP/pyspan/pyspan/config.py:19: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  

EXCLUDING participants who failed the attention check from in-lab data.


/Users/sabinasloman/Box/LoP/pyspan/pyspan/plurals/preprocess.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  demographics.drop_duplicates(inplace = True)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/sabinasloman/Box/LoP/pyspan/pyspan/ratings_task/data.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence th

## Demographics

In [2]:
gender_raw = pd.read_csv("{}in-lab/Gender.csv".format(BASE_DIR))
len(gender_raw), len(gender.loc[gender.ident == "MALE"]), len(gender.loc[gender.ident == "FEMALE"])

(189, 53, 104)

In [3]:
demographic_info(gender.loc[gender.ident == "FEMALE"])

Age: 24.0 (SE = 0.87577981894)
Gender: [('Female', 104)]


In [4]:
demographic_info(gender.loc[gender.ident == "MALE"])

Age: 24.4716981132 (SE = 1.33410215988)
Gender: [('Male', 53)]


### Between subjects

#### 60% item selection threshold

##### Logistic regression

Selection of positive word ~ Participant's gender identity + Condition + Participant's gender identity * Condition + Dummy indicating whether or not this was the first survey the participant took (including participant-level effects)

We hypothesize that the coefficient on Participant's gender identity * Condition will be positive. We will commit to throwing out all data from the gender survey by participants who did not take the survey first if the order dummy has a non-zero coefficient.

In [5]:
gdummied, Y = dummy(gender, [ "MALE", "FEMALE" ],
                    sets = np.stack((positive60, 
                                     negative60)))

In [6]:
X, Y = df_to_matrix(gdummied, Y, columns = { 0: "id",
                                             1: "condition",
                                             2: (0,1), 
                                             3: "order" })

In [7]:
logit = SparseLR(Y, X); print logit.coef[:4]; logit.auc 

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[0. 0. 0. 0.]


0.5

##### t-tests

In [8]:
gsummary = gender[["Condition", "ident"]]
dat = gender[ixs].values
props = np.apply_along_axis(get_prop, 1, dat, positive60,
                            negative60)
gsummary["ppos"] = props
assert gsummary.values.shape == (len(gender), 3)

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


For participants in the male condition, the difference in means mean(% positive words chosen amongst males) - mean(% positive words chosen amongst females) > 0.

In [9]:
a = gsummary.loc[(gsummary["Condition"] == "MALE") & (gsummary["ident"] == "MALE")]["ppos"].values
b = gsummary.loc[(gsummary["Condition"] == "MALE") & (gsummary["ident"] == "FEMALE")]["ppos"].values
stats.ttest_ind(a, b, equal_var = False)

Ttest_indResult(statistic=0.5734117289405796, pvalue=0.5686661620858503)

For participants in the female condition, the difference in means mean(% positive words chosen amongst females) - mean(% positive words chosen amongst males) > 0.

In [10]:
a = gsummary.loc[(gsummary["Condition"] == "FEMALE") & (gsummary["ident"] == "FEMALE")]["ppos"].values
b = gsummary.loc[(gsummary["Condition"] == "FEMALE") & (gsummary["ident"] == "MALE")]["ppos"].values
stats.ttest_ind(a, b, equal_var = False)

Ttest_indResult(statistic=1.161959145849565, pvalue=0.25027571718281966)

The difference in means mean(% positive words chosen amongst those whose condition matched their gender identity) - mean(% positive words chosen amongst those whose condition didn't match their gender identity) > 0

In [11]:
a = gsummary.loc[gsummary["Condition"] == gsummary["ident"]]["ppos"].values
b = gsummary.loc[gsummary["Condition"] != gsummary["ident"]]["ppos"].values

In [12]:
dsw_a = sms.DescrStatsW(a)
dsw_b = sms.DescrStatsW(b)
cm = sms.CompareMeans(dsw_a, dsw_b)
cm.ttest_ind(usevar="unequal", alternative = "larger")

(0.9056742597351738, 0.1832583178516129, 154.70515733051286)

Compute difference in means and a confidence region of +- 2 SEs.

In [13]:
delta = dsw_a.mean - dsw_b.mean
se_delta = cm.std_meandiff_separatevar
print(delta, delta - 2*se_delta, delta + 2*se_delta)

(0.014120436933374836, -0.017061716653509175, 0.04530259052025885)


#### 80% item selection threshold

##### Logistic regression

Selection of positive word ~ Participant's gender identity + Condition + Participant's gender identity * Condition + Dummy indicating whether or not this was the first survey the participant took (including participant-level effects)

In [14]:
gdummied, Y = dummy(gender, sets = np.stack((positive80, 
                                              negative80)),
                    classes = [ "MALE", "FEMALE" ])

In [15]:
X, Y = df_to_matrix(gdummied, Y, columns = { 0: "id",
                                             1: "condition",
                                             2: (0,1), 
                                             3: "order" })

In [16]:
logit = SparseLR(Y, X); print logit.coef[:4]; logit.auc

[0. 0. 0. 0.]


0.5

##### t-tests

In [20]:
gsummary = gender[["Condition", "ident"]]
dat = gender[ixs].values
props = np.apply_along_axis(get_prop, 1, dat, positive80,
                            negative80)
gsummary["ppos"] = props
assert gsummary.values.shape == (len(gender), 3)

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


For participants in the male condition, the difference in means mean(% positive words chosen amongst males) - mean(% positive words chosen amongst females) > 0.

In [21]:
a = gsummary.loc[(gsummary["Condition"] == "MALE") & (gsummary["ident"] == "MALE")]["ppos"].values
b = gsummary.loc[(gsummary["Condition"] == "MALE") & (gsummary["ident"] == "FEMALE")]["ppos"].values
stats.ttest_ind(a, b, equal_var = False)

Ttest_indResult(statistic=1.013491294307676, pvalue=0.3174742983108879)

For participants in the female condition, the difference in means mean(% positive words chosen amongst females) - mean(% positive words chosen amongst males) > 0.

In [22]:
a = gsummary.loc[(gsummary["Condition"] == "FEMALE") & (gsummary["ident"] == "FEMALE")]["ppos"].values
b = gsummary.loc[(gsummary["Condition"] == "FEMALE") & (gsummary["ident"] == "MALE")]["ppos"].values
stats.ttest_ind(a, b, equal_var = False)

Ttest_indResult(statistic=1.3695766379680065, pvalue=0.17679572821015874)

The difference in means mean(% positive words chosen amongst those whose condition matched their gender identity) - mean(% positive words chosen amongst those whose condition didn't match their gender identity) > 0

In [23]:
a = gsummary.loc[gsummary["Condition"] == gsummary["ident"]]["ppos"]
b = gsummary.loc[gsummary["Condition"] != gsummary["ident"]]["ppos"]

In [24]:
dsw_a = sms.DescrStatsW(a)
dsw_b = sms.DescrStatsW(b)
cm = sms.CompareMeans(dsw_a, dsw_b)
cm.ttest_ind(usevar="unequal", alternative = "larger")

(2.440305987849316, 0.007928190321979566, 147.94634628696923)

In [25]:
dsw_a = sms.DescrStatsW(a)
dsw_b = sms.DescrStatsW(b)
cm = sms.CompareMeans(dsw_a, dsw_b)
cm.ttest_ind(usevar="unequal", alternative = "larger")

(2.440305987849316, 0.007928190321979566, 147.94634628696923)

Calculate the difference in means and a confidence region of +- 2 SEs.

In [26]:
delta = dsw_a.mean - dsw_b.mean
se_delta = cm.std_meandiff_separatevar
print(delta, delta - 2*se_delta, delta + 2*se_delta)

(0.0910981476680206, 0.01643689770870138, 0.16575939762733982)


### Within-subjects

#### Logistic regression

Selection of positive word ~ Participant's gender identity + Condition + Participant's gender identity * Condition + Dummy indicating whether or not the participant took the gender survey before the valence survey (including participant-level effects)

We hypothesize that the coefficient on Participant's gender identity * Condition will be positive. 

In [27]:
# Pre-registered: "The within-subject analyses would be run
# both for items within each pre-specified valence category
# (25 positive items, 25 negative items and 25 neutral
# items), and using all 75 non-distractor items."
#
# Change the ixs_ variable below to restrict the subset of
# stims used for analysis
ixs_ = np.arange(100, 125) # Positive items
ixs_ = np.arange(125, 150) # Negative items
ixs_ = np.arange(150, 175) # Neutral items
ixs_ = np.arange(100, 175) # Positive, negative and neutral
# items

In [28]:
gdummied_ws, Y = dummy(gender, 
                       classes = [ "MALE", "FEMALE" ], 
                       within = True, 
                       ixs = ixs_)
X, Y = df_to_matrix(gdummied_ws, Y, 
                    columns = { 0: "id", 1: "condition", 
                                2: (0,1), 3: "order" }, 
                    ixs = ixs_)

In [29]:
logit = SparseLR(Y, X); print logit.coef[:4]; logit.auc

[0.         0.         0.04628291 0.        ]


0.5306000945410451

#### t-tests

In [30]:
gsummary_ws = summarize(gdummied_ws, ixs_)

/Users/sabinasloman/Box/LoP/pyspan/pyspan/plurals/analysis.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  summary["p"] = props


For participants in the male condition, the difference in means mean(% positive words chosen amongst males) - mean(% positive words chosen amongst females) > 0.

In [31]:
stats.ttest_ind(gsummary_ws.loc[(gsummary_ws["Condition"] == 1) & (gsummary_ws["ident"] == 1)]["p"],
                gsummary_ws.loc[(gsummary_ws["Condition"] == 1) & (gsummary_ws["ident"] == 0)]["p"][~np.isnan(gsummary_ws.loc[(gsummary_ws["Condition"] == 1) & (gsummary_ws["ident"] == 0)]["p"])],
                equal_var = False)

Ttest_indResult(statistic=0.3405812551557937, pvalue=0.7362407325351605)

For participants in the female condition, the difference in means mean(% positive words chosen amongst females) - mean(% positive words chosen amongst males) > 0.

In [32]:
stats.ttest_ind(gsummary_ws.loc[(gsummary_ws["Condition"] == 0) & (gsummary_ws["ident"] == 0)]["p"],
                gsummary_ws.loc[(gsummary_ws["Condition"] == 0) & (gsummary_ws["ident"] == 1)]["p"],
                equal_var = False)

Ttest_indResult(statistic=3.0081943370891953, pvalue=0.004581025358360105)

The difference in means mean(% positive words chosen amongst those whose condition matched their gender identity) - mean(% positive words chosen amongst those whose condition didn't match their gender identity) > 0

In [33]:
a = gsummary_ws.loc[gsummary_ws["Condition"] == gsummary_ws["ident"]]["p"]
b = gsummary_ws.loc[gsummary_ws["Condition"] != gsummary_ws["ident"]]["p"][~np.isnan(gsummary_ws.loc[gsummary_ws["Condition"] != gsummary_ws["ident"]]["p"])]

In [34]:
dsw_a = sms.DescrStatsW(a)
dsw_b = sms.DescrStatsW(b)
cm = sms.CompareMeans(dsw_a, dsw_b)
cm.ttest_ind(usevar="unequal", alternative = "larger")

(2.562694517829641, 0.005873878780296829, 109.22185444746182)

Calculate the difference in means and a confidence region of +- 2 SEs.

In [35]:
delta = dsw_a.mean - dsw_b.mean
se_delta = cm.std_meandiff_separatevar
print(delta, delta - 2*se_delta, delta + 2*se_delta)

(0.060666506175188095, 0.013320631937655592, 0.1080123804127206)
